In [ ]:
!pip install evaluate

#### codes are fixed for InferenceModel, all its models, and Visualization Per Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from datasets import load_dataset
import evaluate
import pandas as pd
import torch
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
set_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

class InferenceModel:
    def __init__(self, model_name, device):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", offload_buffers=True)
        self.acc_metric = evaluate.load("accuracy")  
        self.precision_metric = evaluate.load("precision")  
        self.recall_metric = evaluate.load("recall") 
        
        self.df = pd.DataFrame(columns=['pred_answer', 'pred_answertext'])
        self.predictions = []
        self.references = []
        self.answers = []
        self.eval_results = []
        self.model.eval()
        self.logits = []
    def init_eval(self):
        self.acc_metric = evaluate.load("accuracy")  
        self.precision_metric = evaluate.load("precision")  
        self.recall_metric = evaluate.load("recall") 
        self.df = pd.DataFrame(columns=['pred_answer', 'pred_answertext'])
        self.predictions = []
        self.references = []
        self.answers = []
        self.eval_results = []
        
    def get_logprobs(self, prompt):
        inputs = self.tokenizer(prompt, return_tensors="pt")
        outputs = self.model(**inputs, output_hidden_states=True)
        logits = outputs.logits
        # Shape remains, but values are now normalized as log probabilities over the vocabulary.
        logprobs = torch.gather(F.log_softmax(logits, dim=2), 2, output_ids.unsqueeze(2))
        return logprobs, outputs
        
    def inference(self, prompt, max_new_tokens=1):
        """
        Performs inference on a given prompt.
        Returns the decodede output
        """
        generated_text = ''
        len_sentence = 0
        if self.model_name == 'Qwen/Qwen2.5-0.5B-Instruct':
            # WARNING: messages ini harus sama dengan messages di check_index_prompt
            messages = [
            {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
            {"role": "user", "content": prompt}
            ]
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
            
            generated_ids = self.model.generate(
                    **model_inputs,
                    do_sample=False,
                    temperature=None,
                    top_p=None,
                    top_k=None,
                    max_new_tokens=max_new_tokens
                )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]
            generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            len_sentence = len(self.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)["input_ids"])
        elif self.model_name == "google/gemma-2-2b-it":
            # WARNING: messages ini harus sama dengan messages di check_index_prompt
            messages = [
            {"role": "user", "content": prompt}
            ]
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            model_inputs = self.tokenizer([text], return_tensors="pt", add_special_tokens=False).to(self.model.device)
            # print(f"(model_inputs['input_ids'] {(model_inputs['input_ids'])}")
            
            # print(f"len(model_inputs['input_ids'] {len(model_inputs['input_ids'])}")
            sentence_token_texts = self.tokenizer.convert_ids_to_tokens((model_inputs['input_ids'])[0])
            # print(f"sentence_token_texts {sentence_token_texts}")
            
            
            generated_ids = self.model.generate(
                    **model_inputs,
                    do_sample=False,
                    temperature=None,
                    top_p=None,
                    top_k=None,
                    max_new_tokens=max_new_tokens
                )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]
            generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            len_sentence = len(self.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)["input_ids"])
            # print()
            
        else:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
            len_sentence = len(inputs["input_ids"])
            output = self.model.generate(
                **inputs, 
                do_sample=False,
                temperature=None,
                top_p=None,
                top_k=None,
                max_new_tokens=1)
            # print(f"output: {output}")
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output)
            ]
            generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        
        self.answers.append(generated_text)
        # answer = "1" if "option1" in generated_text else "2" if "option2" in generated_text else None
        
        return generated_text, len_sentence

    def batch_inference(self, prompts):
        """
        Perform batch inference.
        """
        inputs = self.tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(self.model.device)
        outputs = self.model.generate(**inputs, max_new_tokens=50)
        generated_texts = []
        for input_ids, output_ids in zip(inputs.input_ids, outputs):
            generated_ids = [
                output_ids[len(input_ids):]
            ]
            generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            generated_texts.append(generated_text)
        self.answers.extend(generated_texts)
        return generated_texts

    def eval(self):
        """
        Evaluates the predictions against the references 
        """
        accuracy_results = self.acc_metric.compute(predictions=self.predictions, references=self.references)

        precision_results = self.precision_metric.compute(predictions=self.predictions, references=self.references, average="macro")

        recall_results = self.recall_metric.compute(predictions=self.predictions, references=self.references, average="macro")

        results = {
            'acc' : accuracy_results,
            'prec' : precision_results,
            'recall' : recall_results
        }
        self.eval_results.append(results)
        print("Evaluation Results:\n", results)
        return results

    
    def parse_save_output(self, generated_text, unique_labels, current_labeltexts, currentlabel):
        """
        generated_text: output from model.generate (e.g. '1', 'option 1', 'anna')
        unique_labels: unique labels from dataset (list of string e.g. ["1", "2"])
        current_labeltexts: the current label options (dict of current label text e.g. {1:'anna', 2:'susan'})
        currentlabel = the current reference label from dataset (string, the dataset['answer']. e.g. "2", "1" )
        """
        pred = None
        clean_generated_text = generated_text.lower().strip()
        for i in unique_labels:
            if str(i) in clean_generated_text or current_labeltexts[int(i)].lower().strip() in clean_generated_text or current_labeltexts[int(i)].lower().removeprefix("the").strip() in clean_generated_text :
                pred = i
                break
        # print(f"gt: {generated_text}")
        # print(f"pred : {pred}")
        if pred == None:
            pred = "9"
        self.predictions.append(pred)
        self.references.append(currentlabel)
        ans_dict = ({'pred_answer':pred, 'pred_answertext' : generated_text})
        # print(f"ans_dict {ans_dict}")
        self.df.loc[len(self.df)] = ans_dict
        return pred



In [ ]:
from huggingface_hub import login

login("***REMOVED***")


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
massive_gemma = InferenceModel("google/gemma-2-2b-it", device)


In [ ]:

massive_qwen = InferenceModel("Qwen/Qwen2.5-0.5B-Instruct", device)

In [ ]:
xcopa_gemma = InferenceModel("google/gemma-2-2b-it", device)


In [ ]:
xcopa_qwen = InferenceModel("Qwen/Qwen2.5-0.5B-Instruct", device)


The neurons we're interested is in act_fn in Qwen2MLP

### start playground

In [ ]:
infer_model = massive_qwen



- ada 2 dict untuk setiap language: 
-> *raw_values_avg_tokens* dan *raw_values_last_token*:

key-nya nama layer dari layer pertama sampe terakhir. setiap valuenya
ukurannya (num_row_in_dataset, hidden_dim) ini neuronnya bisa averaged tokens atau last token. output keluaran suatu neuron adlaah (batch_size, seq_len, num_neurons_di_suatu_layer) tapi seq_len menjadi 1 karena diambil averagenya (raw_values_avg_tokens) atau last token (raw_values_last_token) dari seq) sehingga (num_row_in_dataset, 1, hidden_dim) dijadiin (num_row_in_dataset, hidden_dim) aja.


- tensor *full_raw_values_avg_tokens* dan *full_raw_values_last_token*: adalah tensor dari raw_values_avg_tokens atau raw_values_last_token yang nyatuin semua neurons dari semua layer. ukurannya (batch_size, num_neurons_total_seluruh_layer)


- tensor *full_raw_values*: adalah tensor yang nyatuin full_raw_values_avg_tokens dan full_raw_values_last_token. ukurannya (2, batch_size, num_neurons_total_seluruh_layer). yg pertama full_raw_values_avg_tokens kedua full_raw_values_last_token.


In [ ]:
full_neurons_r = torch.randn(1, 1, 1, 3)
print(full_neurons)
activation_dict = {}

full_neurons = full_neurons_r.transpose(-3,-4)[0].transpose(-1,-2)
ss = (full_neurons.mean(dim=-1).topk(2).indices)
for i in range (full_neurons_r.size(0)):
    activation_dict[i] = ss[i]

activation_dict

In [ ]:
# IMP: ini buat nyatuin tiap entry di dataset
import torch

tensor1 = torch.randn(1, 2, 3)  # Shape: (1, 2, 3)
print(tensor1)
tensor2 = torch.randn(2, 2, 3)  # Shape: (2, 2, 3)
print(tensor2)
# Concatenate along dim=0
result = torch.cat([tensor1, tensor2], dim=0)

print(result.shape)  # Output: (3, 2, 3)


In [ ]:
tensor2 = torch.tensor([[1, 2], [3, 4]])  # Shape: [1, 2]
# print(tensor2.float().mean(dim=1))
print(tensor2.float()[-1, :])

In [ ]:
tensor2 = torch.tensor([[1, 2], [3, 4]])  # Shape: [1, 2]
print(tensor2.shape)
lists = [tensor2, tensor2, tensor2]
stacked = torch.stack(lists, dim=0)
print(stacked.shape)

In [ ]:
raw_values_avg_tokens = {}
raw_values_last_token = {}
# ini yg raw value


# def aggregate_across_tokens(output, is_averaged_tokens):
#     if is_averaged_tokens:
#         output = output.float().mean(dim=1, keepdim=True) 
#     else:
#         output = output.float()[:, -1:] # get the last token
#     return output

def save_raw_vals_to_dict(name, dictname, savees):
    if not name in dictname:
        dictname[name] = savees
    else:
        dictname[name] = torch.cat([dictname[name], savees], dim=0)

def coba(name, is_averaged_tokens, output):
    # output is always (batch size, 1, hidden dim)
    # print(f"Layer: {name}, output Shape: {output.shape}")
    # print("Output", output)
    # output is always (batch size, hidden dim), where hidden dim is the number of neurons in a layer.
    output 
    if is_averaged_tokens: # simpen average value across tokens
        output = output.float().mean(dim=1) 
        save_raw_vals_to_dict(name, raw_values_avg_tokens, output)
    else:
        output = output.float()[-1,:] # simpen the last token
        save_raw_vals_to_dict(name, raw_values_last_token, output)
        
    

In [ ]:
x = torch.tensor([[[1, 2], [3, 4]]])
y = torch.tensor([[[5, 6], [7, 8]]])
z = torch.tensor([[[1, 6], [3, 2]]])

# layer_outputs['a'] = x
coba("a", True, x)
coba("a", True, y)
print(raw_values_avg_tokens['a'])
print(raw_values_avg_tokens['a'].shape)

coba("a", False, y)
coba("a", False, z)
print(raw_values_last_token['a'])
print(raw_values_last_token['a'].shape)


In [ ]:
x = torch.tensor([[[1, 2], [3, 4]]])
# layer_outputs['a'] = x
coba("b", True, y)
coba("b", True, y)
print(raw_values_avg_tokens['b'])
print(raw_values_avg_tokens['b'].shape)

coba("b", False, x)
coba("b", False, x)

print(raw_values_last_token['b'])
print(raw_values_last_token['b'].shape)


In [ ]:
list(raw_values_avg_tokens.values())

In [ ]:
# IMP: nyatuin tiap layer OKE
full_raw_values_avg_tokens = list(raw_values_avg_tokens.values())[0]
full_raw_values_last_token = list(raw_values_last_token.values())[0]

# print(full_raw_values_avg_tokens.shape)
# print(full_raw_values_last_token.shape)
for i in list(raw_values_avg_tokens.values())[1:]:
    # print(i.shape)
    full_raw_values_avg_tokens = torch.cat([full_raw_values_avg_tokens, i], dim=-1)
for i in list(raw_values_last_token.values())[1:]:
    full_raw_values_last_token = torch.cat([full_raw_values_last_token, i], dim=-1)

In [ ]:
print(full_raw_values_avg_tokens)
print(full_raw_values_last_token)


In [ ]:
full_raw_values_last_token.shape

In [ ]:
# IMP: nyatuin avg dan last
full_raw_values = torch.stack((full_raw_values_avg_tokens, full_raw_values_last_token), dim=0)
# stack buat nyatuin avg token dan last token OKE
# torch.stack((raw_values_last_token['a'], raw_values_avg_tokens['a']), dim=0)
full_raw_values

In [ ]:
full_raw_values.shape

In [ ]:
ex = torch.tensor(
    [[[[0, 1, 2, 3],
        [4, 5, 6, 7]]]]
)
ex.transpose(-1,-2)

### === end playground

### prompt dengan predict intent

In [ ]:

def check_index_prompt(when, prompt, model_name): # TODO: method ini pindahin ke class infer_model dan hapus printnya 
    """
    return len tokenized text when it is cut (excluding the chat template), len tokenized text 
    """
    
    if model_name == 'Qwen/Qwen2.5-0.5B-Instruct':
        messages = [
            {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
            {"role": "user", "content": prompt}
            ]
        text = infer_model.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        sentence_tokens = infer_model.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
        sentence_ids = sentence_tokens["input_ids"]
        sentence_token_texts = infer_model.tokenizer.convert_ids_to_tokens(sentence_ids)[:-5]
        # print(f"{when} sentence_token_texts {sentence_token_texts}. len(sentence_token_texts) {len(sentence_token_texts)}")
        return len(sentence_token_texts), len(sentence_ids)
    elif model_name == 'google/gemma-2-2b-it':
        messages = [
            {"role": "user", "content": prompt}
            ]
        text = infer_model.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        sentence_tokens = infer_model.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
        sentence_ids = sentence_tokens["input_ids"]
        sentence_token_texts = infer_model.tokenizer.convert_ids_to_tokens(sentence_ids)[:-5]
        # print(f"{when} sentence_token_texts {sentence_token_texts}. len(sentence_token_texts) {len(sentence_token_texts)}")
        return len(sentence_token_texts), len(sentence_ids)


def get_index_start_end_prompt(data, model_name, dataset_name): # TODO: method ini pindahin ke class infer_model dan hapus printnya 
    """
    return id start, id end, len sentence tokenize, prompt whole
    buat massive
    """
    prompt_before = prompt_after = prompt_whole = ""
    if dataset_name == "AmazonScience/massive":
        utterance, options = data
        prompt_before = (
            f"""
            Instruction: Classify the intent of the following utterance.
            Utterance: 
            """
            )
        prompt_after = (
            f"""
            Instruction: Classify the intent of the following utterance.  
            Utterance: {utterance}.  
            """
            )
        prompt_whole = (
            f"""
            Instruction: Classify the intent of the following utterance.  
            Utterance: {utterance}.  
            Options: {options}. 
            Intent: 
            """
            )
    elif dataset_name == "xcopa":
        premise, choice1, choice2, question_type = data
        prompt_before = (
            f"""
            Premise: 
            """)
        prompt_after = (
            f"""
            Premise: {premise}
            """)
        prompt_whole = (
            f"""
            Premise: {premise}
            I'm hesitating between the two options. Help me choose the {question_type}: 
            - {choice1} 
            - {choice2}
            """)
    if prompt_before == "":
        print("dataset invalid")
    id_prompt_start, _ = check_index_prompt("prompt_before", prompt_before, model_name)
    id_prompt_end, _ = check_index_prompt("prompt_after", prompt_after, model_name)
    _, len_sentence = check_index_prompt("prompt_whole", prompt_whole, model_name)
    return id_prompt_start, id_prompt_end, len_sentence, prompt_whole


In [ ]:
import psutil

def check_cpu_memory():
    mem = psutil.virtual_memory()
    print(f"Total Memory: {mem.total / 1024**3:.2f} GB")
    print(f"Available Memory: {mem.available / 1024**3:.2f} GB")
    print(f"Used Memory: {mem.used / 1024**3:.2f} GB")
    print(f"Memory Usage: {mem.percent}%")
check_cpu_memory()

In [ ]:
!nvidia-smi

# Per Model Visualization

In [ ]:
num_layer_gemma2 = 26 
num_layer_qwen05 = 24

### Visualize distribution

In [ ]:
import torch 
from datasets import get_dataset_config_names
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import scipy.stats as stats


def hist_neuron_values(num_neurons, num_layer, data, label=""):
    """
    data isinya untuk seluruh neuron di 1 data point 1 bahasa. co: full_languages_raw_values[0][0][0]
    """
    
    num_points = num_neurons  # 116736
    step_size = num_neurons / num_layer
    # step_size = 24
    x_indices = np.arange(0, num_points, step_size)  
    x_labels = [str(i) for i in x_indices]  
    
    # Plot the data
    plt.figure(figsize=(7, 2))
    plt.plot(data.numpy(), label="Tensor Values", linewidth=0.8)  
    plt.xticks(x_indices, x_labels, rotation=45) 
    plt.xlabel("Index")
    plt.ylabel("Value")
    plt.title(f"Tensor Value Distribution {label}")
    plt.legend()
    plt.show()



def kstest(tensor):
    """
    tensornya isi untuk seluruh neuron di 1 data point 1 bahasa. co: full_languages_raw_values[0][0][0]
    """
    # List of distributions to test
    distributions = ["norm", "expon", "gamma", "lognorm", "beta", "weibull_min"]
    
    best_fit = None
    best_pvalue = 0
    
    for dist_name in distributions:
        dist = getattr(stats, dist_name)
        params = dist.fit(tensor)  # Fit distribution to data
        stat, p_value = stats.kstest(tensor, dist_name, args=params)  # KS test
    
        print(f"{dist_name}: p-value = {p_value:.4f}")
    
        if p_value > best_pvalue:
            best_pvalue = p_value
            best_fit = dist_name
    
    print(f"\nBest fitting distribution: {best_fit}")
# kstest(full_languages_raw_values[0][0][0])


def plot_kde(tensor):
    """
    tensornya isi untuk seluruh neuron di 1 data point 1 bahasa. co:
    full_languages_raw_values[0][0][0]
    """
    sns.kdeplot(tensor, bw_adjust=0.5)
    plt.xlabel("Value")
    plt.ylabel("Density")
    plt.title("Tensor Value Distribution (KDE)")
    plt.show()

def get_language_dict(dataset_name):
    dataset_name = dataset_name
    configs = get_dataset_config_names(dataset_name, trust_remote_code=True)
    n_instances = 0
    n_lang = 0
    language_dict = {}
    # register_hook(infer_model, handlers)
    
    for lang in enumerate([lang for lang in configs if not lang.startswith("all") and not lang.startswith("translation")]):
        # if n_lang >= 2:
        language_dict[lang[0]] = lang[1]
        # print(lang)
    
    return language_dict


def clear_cuda_cache():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()


def show_stats(tensor):
    """
    tensor:full_languages_raw_values[lang1][0][0] / 1 language semua neuron dan rows
    """
    print("Mean:", torch.mean(tensor).item())
    print("Median:", torch.median(tensor).item())
    print("Standard Deviation:", torch.std(tensor).item())
    print("Min:", torch.min(tensor).item())
    print("Max:", torch.max(tensor).item())
    
    # Mode Calculation
    mode_value, mode_count = torch.mode(tensor)
    print("Mode:", mode_value.item())

def visualize_stats(tensor, a_lang, a_row=0):
    """
    tensor: full raw values dim 4
    a_lang: index of language to visualize
    """
    tensor_lang_row = tensor[a_lang][0][a_row]
    show_stats(tensor_lang_row)
    plot_kde(tensor_lang_row)
    kstest(tensor_lang_row)
    
def compare_hist(tensor, num_layer, lang1, row1, lang2, row2, lang_dict, top=10):
    """
    tensor: full raw values dim 4
    a_lang: index of language to visualize
    """


    
    tensor1a = tensor[lang1][0][row1]
    tensor1b = tensor[lang1][0][row2]

    tensor2a = tensor[lang2][0][row1]
    tensor2b = tensor[lang2][0][row2]
    
    hist_neuron_values(tensor.size(-1), num_layer, tensor1a, f"{row1} in {lang_dict[lang1]}")
    print(f"topk {row1} in {lang_dict[lang1]}: {tensor1a.topk(top)}")
    hist_neuron_values(tensor.size(-1), num_layer, tensor1b, f"{row2} in {lang_dict[lang1]}")
    print(f"topk {row2} in {lang_dict[lang1]}: {tensor1b.topk(top)}")

    hist_neuron_values(tensor.size(-1), num_layer, tensor2a, f"{row1} in {lang_dict[lang2]}")
    print(f"topk {row1} in {lang_dict[lang2]}: {tensor2a.topk(top)}")

    hist_neuron_values(tensor.size(-1), num_layer, tensor2b, f"{row2} in {lang_dict[lang2]}")
    print(f"topk {row2} in {lang_dict[lang2]}: {tensor2b.topk(top)}")

massive_lang_dict = get_language_dict("AmazonScience/massive")
xcopa_lang_dict = get_language_dict("xcopa")


#### 🐸Massive Qwen

In [ ]:
full_languages_raw_values = torch.load("/kaggle/input/raw-qwen05-predict-200/raw_qwen05_predict_massive200_selected_v.pt", weights_only=False)
print(full_languages_raw_values.shape)
a_lang = 0
print(f"lang 0")
visualize_stats(full_languages_raw_values, a_lang, a_row=0)
compare_hist(full_languages_raw_values, num_layer=24, lang1=0, row1=0, lang2=19, row2=199, lang_dict=massive_lang_dict)

#### 🐸XCOPA Gemma

In [ ]:
full_languages_raw_values_xcopa_gemma = torch.load("/kaggle/input/raw-gemma2-predictv1/raw_gemma2_predict_xcopa200.pt", weights_only=False)
# full_languages_raw_values_xcopa_gemma.shape
# print(full_languages_raw_values_xcopa_gemma.shape)
a_lang = 0
print(f"lang 0")

visualize_stats(full_languages_raw_values_xcopa_gemma, a_lang, a_row=0)
compare_hist(full_languages_raw_values_xcopa_gemma, num_layer=26, lang1=0, row1=0, lang2=10, row2=19, lang_dict=xcopa_lang_dict)

#### 🐸XCOPA Qwen

In [ ]:
full_languages_raw_values_xcopa_qwen = torch.load("/kaggle/input/raw-qwen05-predict-200/raw_qwen05_predict_xcopa200.pt", weights_only=False)
print(full_languages_raw_values_xcopa_qwen.shape)
a_lang = 0
print(f"lang 0")

visualize_stats(full_languages_raw_values_xcopa_qwen, a_lang, a_row=0)
compare_hist(full_languages_raw_values_xcopa_qwen, num_layer=num_layer_qwen05, lang1=0, row1=0, lang2=10, row2=19, lang_dict=xcopa_lang_dict)

#### 🐸Massive Gemma

In [ ]:
full_languages_raw_values_massive_gemma = torch.load("/kaggle/input/raw-gemma2-predictv1/raw_gemma2_predict_massive200.pt", weights_only=False)
print(full_languages_raw_values_massive_gemma.shape)
a_lang = 0
print(f"lang 0")
visualize_stats(full_languages_raw_values_massive_gemma, a_lang, a_row=0)
compare_hist(full_languages_raw_values_massive_gemma, num_layer=num_layer_gemma2, lang1=0, row1=0, lang2=19, row2=199, lang_dict=massive_lang_dict)

### Pairing antar language

#### Kesamaan index neuron yang teraktivasi.
- Case 1: Neuron yang teraktivasi -> yang actv. valuenya di atas 0 untuk seluruh data point di dataset
- Case 2: Neuron yang teraktivasi -> anggap abs(actv_value) > 0.5 = 1 -> neuron yang probability munculnya > meannya itu yang teraktivasi

#### ===== activation value dengan prompt tanpa predict (fully in their own languages)

case 1

di sini anggap activated neuron (avg token) yang di setiap row di dataset(cuman 2 row) actv valuenya > 0

In [ ]:
import numpy as np
from scipy.stats import entropy
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


def get_k_lang_actv_dict(k, full_neurons, method="default", topk=0):
    
    """
    di sini anggap activated neuron (avg token) yang di setiap row di dataset(cuman 2 row) actv valuenya > 0.
    k = num language"""
    activation_dict = {}
    
    full_neurons = full_neurons.transpose(-3,-4)[0].transpose(-1,-2)

    if method == "default":
        for i in range (full_neurons.size(0)):
            tensor_lang = full_neurons[i]
            rows_with_both_positive = (tensor_lang > 0).all(dim=-1)
            
            indices = torch.where(rows_with_both_positive)[0]
            activation_dict[i] = indices
            # indices.shape
            # print(indices.shape)
    elif method == "topk":
        if topk==0:
            print(f"topk must not be 0")
        top = (full_neurons.mean(dim=-1).topk(topk).indices)
        for i in range (full_neurons.size(0)):
            activation_dict[i] = top[i]
    
    return activation_dict

# Step 1: Convert lists to probability distributions
def to_probability_distribution(values):
    total = sum(values)
    return [v / total for v in values]


def make_heatmap_neuron_overlap(activation_dict, k, with_label=True, method="default", alpha=1, normalized =False):    
    # Example dictionary: keys 0-52, values are 1D tensors of activated neuron indices
    # activation_dict = get_k_lang_actv_dict(10)

    overlap_matrix = torch.tensor([])
    if method == "default":
        # Step 1: Create a binary matrix
        max_neuron_index = max(max(indices) for indices in activation_dict.values()) + 1  # Find the maximum neuron index
        binary_matrix = torch.zeros((k+1, max_neuron_index), dtype=torch.int)  # Initialize binary matrix
        
        for key, indices in activation_dict.items():
            binary_matrix[key, indices] = 1  # Set activated neurons to 1
        
        # Step 2: Compute overlaps (dot product between rows)
        overlap_matrix = torch.matmul(binary_matrix, binary_matrix.T)  # Dot product of binary_matrix with its transpose

    elif method == "jaccard":
        max_neuron_index = max(max(indices) for indices in activation_dict.values()) + 1
        binary_matrix = torch.zeros((k+1, max_neuron_index), dtype=torch.int)
    
        # Fill binary matrix with activation data
        for key, indices in activation_dict.items():
            binary_matrix[key, indices] = 1  
    
        # Compute Jaccard distance matrix
        overlap_matrix = torch.zeros((k+1, k+1))
    
        for i in range(k+1):
            for j in range(k+1):
                intersection = (binary_matrix[i] & binary_matrix[j]).sum().item()
                union = (binary_matrix[i] | binary_matrix[j]).sum().item()
                jaccard_similarity = intersection / union if union > 0 else 0
                overlap_matrix[i, j] = jaccard_similarity
        overlap_matrix = overlap_matrix ** alpha
        if normalized:
            overlap_matrix = overlap_matrix / overlap_matrix.sum(axis=1, keepdims=True)

    
    # Step 3: Visualize the heatmap
    plt.figure(figsize=(10, 8))
    if with_label:
        sns.heatmap(overlap_matrix.numpy(), annot=True, fmt="d", cmap="YlOrRd", 
                    xticklabels=range(k+1), yticklabels=range(k+1))
    else:
        sns.heatmap(overlap_matrix.numpy(), fmt="d", cmap="YlOrRd",cbar=True)
    plt.xlabel("Key")
    plt.ylabel("Key")
    plt.title(f"Overlap Heatmap of Activated Neurons: {method}")
    plt.show()
    return overlap_matrix


def normed_heatmap_neuron_overlap(num_lang, activation_dict):
    # Step 1: Create a binary matrix
    max_neuron_index = max(max(indices) for indices in activation_dict.values()) + 1  # Find the maximum neuron index
    binary_matrix = torch.zeros((num_lang, max_neuron_index), dtype=torch.int)  # Initialize binary matrix
    
    for key, indices in activation_dict.items():
        binary_matrix[key, indices] = 1  # Set activated neurons to 1
    
    # Step 2: Compute overlaps (dot product between rows)
    overlap_matrix = torch.matmul(binary_matrix, binary_matrix.T)  # Dot product of binary_matrix with its transpose
    
    # Step 3: Normalize the overlap matrix
    # Compute the number of activated neurons for each key
    num_activated_neurons = binary_matrix.sum(dim=1, keepdim=True)
    
    # Normalize by the minimum number of activated neurons between each pair of keys
    normalized_overlap_matrix = overlap_matrix / torch.minimum(
        num_activated_neurons, num_activated_neurons.T
    )
    
    # Ensure the diagonal is exactly 1 (self-overlap is always 1)
    normalized_overlap_matrix.fill_diagonal_(1)
    
    # Step 4: Visualize the normalized heatmap (no labels or annotations)
    plt.figure(figsize=(10, 8))
    sns.heatmap(normalized_overlap_matrix.numpy(), cmap="YlOrRd", cbar=True, vmin=0, vmax=1)
    plt.xlabel("Key")
    plt.ylabel("Key")
    plt.title("Normalized Overlap Heatmap of Activated Neurons")
    plt.show()

#### =====activation value dengan prompt predict (prompts in english)

#### Visualisasi distribusi neurons 

lihat distribusi nilai neurons 

very similar neurons activated for prompts in english for every language

-> may need to sort out the specific only to the pair of language

case 1

In [ ]:
def visualize_overlap(num_lang , tensor, method="default", topk=0):
    """
    tensor: full neurons 4 dim
    """
    activation_dict = get_k_lang_actv_dict(num_lang, tensor, method, topk)
    make_heatmap_neuron_overlap(activation_dict, num_lang, False)
    normed_heatmap_neuron_overlap(num_lang, activation_dict)
    make_heatmap_neuron_overlap(activation_dict, num_lang, False, "jaccard", 2)


#### 🐸Massive Qwen

In [ ]:
num_lang, _, num_rows, num_neurons = full_languages_raw_values.shape
visualize_overlap(num_lang, full_languages_raw_values)

#### 🐸XCOPA Gemma

In [ ]:
num_lang, _, num_rows, num_neurons = full_languages_raw_values_xcopa_gemma.shape
visualize_overlap(num_lang, full_languages_raw_values_xcopa_gemma)

#### 🐸XCOPA Qwen

In [ ]:
num_lang, _, num_rows, num_neurons = full_languages_raw_values_xcopa_qwen.shape
visualize_overlap(num_lang, full_languages_raw_values_xcopa_qwen)

#### 🐸Massive Gemma

In [ ]:
num_lang, _, num_rows, num_neurons = full_languages_raw_values_massive_gemma.shape
visualize_overlap(num_lang, full_languages_raw_values_massive_gemma)

## Overlap on same model different dataset 

In [ ]:
# First dictionary with full language names and codes
languages = massive_lang_dict

# Second dictionary with language codes
second_dict = xcopa_lang_dict

# Extract language codes from `languages`
lang_code_dict = {k: v.split('(')[-1].split('-')[0] for k, v in languages.items()}

# Find matching languages
matching_languages = {key: value for key, value in second_dict.items() if value in lang_code_dict.values()}

# Get indices of matching languages in first dictionary
id_match_massive = [key for key, value in lang_code_dict.items() if value in matching_languages.values()]

# Get indices of matching languages in second dictionary
id_match_xcopa = list(matching_languages.keys())

# Print results
print("Indices in first dictionary:", id_match_massive)
print("Indices in second dictionary:", id_match_xcopa)


In [ ]:
id_match_massive = id_match_massive[:-1]
id_match_massive

In [ ]:
import matplotlib.pyplot as plt
import torch

# Example data (replace with your actual data)
data = torch.randint(0, 116736, (1000,)).tolist()  # Random data between 0 and 116736

# Define bins with step size 4864
bin_width = 116736 / 24
bins = torch.arange(0, 116736 + bin_width, bin_width).tolist()  # Create bin edges

# Plot histogram
plt.hist(data, bins=bins, edgecolor='black', alpha=0.7)
plt.xlabel("Range (bin size = 4864)")
plt.ylabel("Frequency")
plt.title("Histogram with Bin Size 4864")
plt.show()


In [ ]:
max([2,3,4])

In [ ]:
import matplotlib.pyplot as plt
import torch

def overlap_model_diff_dataset(massive_dict, xcopa_dict, num_neurons, num_layer):
    """
    key in massive_dict is always already in xcopa_dict
    """
    overlap_dict = {key: list(set(massive_dict[key].tolist()) & set(xcopa_dict[key].tolist())) for key in massive_dict.keys() if key in xcopa_dict}
    overlap_proportion_dict = dict()
    for key, value in overlap_dict.items():
        max_overlap = max([len(massive_dict[key]), len(xcopa_dict[key])])
        lang = massive_lang_dict[id_match_massive[key]]
        overlap_proportion_dict[lang] = len(value)/max_overlap
        label = f"overlap {lang} massive-xcopa"
        bin_width = num_neurons / num_layer
        bins = torch.arange(0, num_neurons + bin_width, bin_width).tolist()  # Create bin edges
        plt.figure(figsize=(6, 4))  
        # Plot histogram
        plt.hist(value, bins=bins, edgecolor='black', alpha=0.7)
        plt.xlabel("Index")
        plt.ylabel("Frequency")
        plt.title(f"{label}")
        plt.show()
    print(overlap_proportion_dict)
    return overlap_dict
        
    # print(overlap_proportion_dict)
        # hist_neuron_values(num_neurons, num_layer, torch.tensor(value), label)


### 🐸Qwen 0.5 on Massive and XCOPA

In [ ]:
num_lang, _, _, num_neurons = full_languages_raw_values.shape
activation_dict = get_k_lang_actv_dict(52, full_languages_raw_values )
massive_qwen_dict = {i: activation_dict[k] for i, k in enumerate(id_match_massive) if k in activation_dict}
activation_dict = get_k_lang_actv_dict(11, full_languages_raw_values_xcopa_qwen)
xcopa_qwen_dict = {i: activation_dict[k] for i, k in enumerate(id_match_xcopa) if k in activation_dict}
overlap_dict = overlap_model_diff_dataset(massive_qwen_dict, xcopa_qwen_dict, num_neurons, num_layer_qwen05)
torch.save(overlap_dict, "qwen05-overlap-across-dataset")

### 🐸Gemma 2 on Massive and XCOPA

In [ ]:
num_lang, _, _, num_neurons = full_languages_raw_values_massive_gemma.shape

activation_dict = get_k_lang_actv_dict(52, full_languages_raw_values_massive_gemma)
massive_gemma_dict = {i: activation_dict[k] for i, k in enumerate(id_match_massive) if k in activation_dict}
activation_dict = get_k_lang_actv_dict(11, full_languages_raw_values_xcopa_gemma)
xcopa_gemma_dict = {i: activation_dict[k] for i, k in enumerate(id_match_xcopa) if k in activation_dict}
overlap_dict = overlap_model_diff_dataset(massive_gemma_dict, xcopa_gemma_dict, num_neurons, num_layer_gemma2)
torch.save(overlap_dict, "gemma2-overlap-across-dataset")

In [ ]:
check_cpu_memory()

In [ ]:
import torch
import torch.nn.functional as F
import torch
import scipy.stats


def dist_matrix(tipe, tensor, num_lang, with_label, lang_dict, threshold):
    """
    tipe = kl or spearman 
    tensor: shape(num_lang, 2, num_rows, num_neurons)
    num_lang: num of languages
    with_label: boolean
    threshold: (float) e.g. 0.9, the threshold of correlation coeff. that determines if 2 languages overlap
    lang_dict: (dict) e.g. {0: "AF-za", }
    """
    matrix = torch.zeros(num_lang, num_lang)
    if tipe == "kl":
        activation_tensor = tensor.transpose(0, 1)[0].mean(dim=-2).clamp(min=0)
        eps = 1e-6  # Small epsilon to avoid division by zero
        
        # Normalize each row to sum to 1 (convert to probability distributions)
        row_sums = activation_tensor.sum(dim=1, keepdim=True)
        activation_tensor = activation_tensor / (row_sums + eps)
        
        # Handle zero rows (replace with uniform distribution)
        zero_rows = row_sums.squeeze() == 0
        activation_tensor[zero_rows] = 1.0 / activation_tensor.shape[1]
        
        
        for i in range(num_lang):
            for j in range(num_lang):
                matrix[i, j] = F.kl_div(
                    activation_tensor[j].clamp(min=eps).log(),  # Q (log-probabilities)
                    activation_tensor[i],  # P (reference distribution)
                    reduction="batchmean",  # Use batchmean instead of sum
                    log_target=False  # P is not in log-space
                )
    elif tipe =="spearman_avg_on_activation_value" : 
        activation_tensor = tensor.transpose(0,1)[0].mean(dim=-2)
        
        # Compute Spearman's rho for each pair of rows
        for i in range(num_lang):
            for j in range(num_lang):
                rho, _ = scipy.stats.spearmanr(activation_tensor[i].cpu().numpy(), activation_tensor[j].cpu().numpy())
                matrix[i, j] = rho  # Store result
                
    elif tipe == "spearman_avg_on_corr_value" :
        activation_tensor = tensor.transpose(0,1)[0].transpose(0,1)
        num_data_points = activation_tensor.size(0)
        assert num_data_points == 200, "not 200"
        #(num_data_points, num_lang, num_features)
        
        # Compute and accumulate Spearman correlation matrices
        for data_idx in range(num_data_points):
            activations = activation_tensor[data_idx].cpu().numpy()  # (num_lang, num_features)
            
            # Compute Spearman correlation matrix for this data point
            corr_matrix, _ = scipy.stats.spearmanr(activations, axis=1)
            
            # Accumulate
            matrix += corr_matrix
        
        # Average across all data points
        matrix /= num_data_points   
    else:
        print("tipe doesnt exist")
        return

    plt.figure(figsize=(10, 8))
    if with_label:
        sns.heatmap(matrix.numpy(), annot=True, fmt="d", cmap="YlOrRd", 
                    xticklabels=range(k+1), yticklabels=range(k+1))
    else:
        sns.heatmap(matrix, fmt="d", cmap="YlOrRd",cbar=True)
    plt.xlabel("Key")
    plt.ylabel("Key")
    plt.title(f"Overlap Heatmap of Activated Neurons: {tipe}")
    plt.show()

    # if tipe is correlation, return dict of each languages' closest overlap
    if tipe.startswith("spearman"):
        lang_correlation_dict = dict()
        for i in range(num_lang):  # Iterate over each language (row index)
            # Find indices where correlation >= threshold (excluding self)
            mask = (matrix[i] >= threshold) & (torch.arange(num_lang) != i)
            correlated_indices = torch.where(mask)[0]  # Indices of correlated languages
            correlated_values = matrix[i, correlated_indices]  # Corresponding correlation values
            correlated_list = list(zip(correlated_indices.tolist(), correlated_values.tolist()))
            correlated_list.sort(key=lambda x: x[1], reverse=True)
            # Convert to list of (index, correlation coefficient) tuples
            lang = lang_dict[i] 
            lang_correlation_dict[lang] = correlated_list
        return lang_correlation_dict
                    
    

In [ ]:
def visualize_corr(tensor, num_lang, lang_dict, threshold=0.7):
    # dist_matrix("kl", tensor, num_lang, False, lang_dict, threshold)
    spearman1 = dist_matrix("spearman_avg_on_activation_value", tensor, num_lang, False, lang_dict, threshold)
    spearman2 = dist_matrix("spearman_avg_on_corr_value", tensor, num_lang, False, lang_dict, threshold)
    return spearman1, spearman2

#### 🐸Massive Qwen

In [ ]:
num_lang, _, num_rows, num_neurons = full_languages_raw_values.shape
s1,s2 = visualize_corr(tensor=full_languages_raw_values, num_lang=num_lang, lang_dict=massive_lang_dict, threshold=0.7)
torch.save(s2, "lang_overlap_massive_qwen05")

#### 🐸XCOPA Gemma

In [ ]:
num_lang, _, num_rows, num_neurons = full_languages_raw_values_xcopa_gemma.shape
s1,s2 = visualize_corr(tensor=full_languages_raw_values_xcopa_gemma, num_lang=num_lang, lang_dict=xcopa_lang_dict, threshold=0.7)
torch.save(s2, "lang_overlap_xcopa_gemma2")

### 🐸XCOPA Qwen

In [ ]:
num_lang, _, num_rows, num_neurons = full_languages_raw_values_xcopa_qwen.shape
s1,s2 = visualize_corr(tensor=full_languages_raw_values_xcopa_qwen, num_lang=num_lang, lang_dict=xcopa_lang_dict, threshold=0.7)
torch.save(s2, "lang_overlap_xcopa_qwen05")

### 🐸MASSIVE Gemma

In [ ]:
num_lang, _, num_rows, num_neurons = full_languages_raw_values_massive_gemma.shape
s1,s2 = visualize_corr(tensor=full_languages_raw_values_massive_gemma, num_lang=num_lang, lang_dict=massive_lang_dict, threshold=0.7)
torch.save(s2, "lang_overlap_massive_gemma2")

In [ ]:
lihat = torch.load("/kaggle/working/lang_overlap_massive_qwen05", weights_only=False)
lihat['id-ID']

In [ ]:
lihat = torch.load("/kaggle/working/lang_overlap_massive_gemma2", weights_only=False)
lihat['id-ID']

In [ ]:
lihat = torch.load("/kaggle/working/lang_overlap_xcopa_gemma2", weights_only=False)
# lihat['id-ID'][:5]
lihat

In [ ]:
lihat = torch.load("/kaggle/working/lang_overlap_xcopa_qwen05", weights_only=False)
# lihat['id-ID'][:5]
lihat

In [ ]:
import os
import json
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['KAGGLE_KEY'] = user_secrets.get_secret("KAGGLE_KEY")
os.environ['KAGGLE_USERNAME'] = user_secrets.get_secret("KAGGLE_USERNAME")


In [ ]:
os.makedirs('/kaggle/dataset', exist_ok=True)

os.makedirs('/kaggle/dataset/overlap-spearman', exist_ok=True)

# Change below
meta = dict(
    id="inayarhmns/overlap-spearman-langs-neurons",
    title="overlap-spearman-langs-neurons",
    isPrivate=True,
    licenses=[dict(name="other")]
)
with open(f'/kaggle/dataset/overlap-spearman/dataset-metadata.json', 'w') as f:
    json.dump(meta, f)


In [ ]:
!cp /kaggle/working/lang_overlap_massive_gemma2 /kaggle/dataset/overlap-spearman
!cp /kaggle/working/lang_overlap_massive_qwen05 /kaggle/dataset/overlap-spearman
!cp /kaggle/working/lang_overlap_xcopa_gemma2 /kaggle/dataset/overlap-spearman
!cp /kaggle/working/lang_overlap_xcopa_qwen05 /kaggle/dataset/overlap-spearman


In [ ]:
torch.save(massive_lang_dict, "/kaggle/dataset/overlap-spearman/massive_lang_dict")

In [ ]:
torch.save(xcopa_lang_dict, "/kaggle/dataset/overlap-spearman/xcopa_lang_dict")

In [ ]:
!ls -lh /kaggle/dataset/overlap-spearman


In [ ]:
!kaggle datasets create -p "/kaggle/dataset/overlap-spearman" --dir-mode zip

case 2

### 🐸Massive Qwen

In [ ]:
num_lang, _, num_rows, num_neurons = full_languages_raw_values.shape
visualize_overlap(num_lang, full_languages_raw_values, method="topk", topk=1000)

TODO

### LAPE

In [ ]:
def LAPE(activation_probs, top_rate = 0.01,filter_rate=0.95,activation_bar_ratio=0.95):
    """    
    activation_probs = # layer x inter x lang_num
    """    
    num_layers = activation_probs.size(0)
    normed_activation_probs = activation_probs / activation_probs.sum(dim=-1, keepdim=True)
    normed_activation_probs[torch.isnan(normed_activation_probs)] = 0
    log_probs = torch.where(normed_activation_probs > 0, normed_activation_probs.log(), 0)
    entropy = -torch.sum(normed_activation_probs * log_probs, dim=-1)
    largest = False
    
    if torch.isnan(entropy).sum():
        print(torch.isnan(entropy).sum())
        raise ValueError
    
    flattened_probs = activation_probs.flatten()
    top_prob_value = flattened_probs.kthvalue(round(len(flattened_probs) * filter_rate)).values.item()
    print(top_prob_value)
    # dismiss the neruon if no language has an activation value over top 90%
    top_position = (activation_probs > top_prob_value).sum(dim=-1)
    entropy[top_position == 0] = -torch.inf if largest else torch.inf

    flattened_entropy = entropy.flatten()
    top_entropy_value = round(len(flattened_entropy) * top_rate)
    _, index = flattened_entropy.topk(top_entropy_value, largest=largest)
    row_index = index // entropy.size(1)
    col_index = index % entropy.size(1)
    selected_probs = activation_probs[row_index, col_index] # n x lang
    # for r, c in zip(row_index, col_index):
    #     print(r, c, activation_probs[r][c])

    # print(selected_probs.size(0), torch.bincount(selected_probs.argmax(dim=-1)))
    selected_probs = selected_probs.transpose(0, 1)
    activation_bar = flattened_probs.kthvalue(round(len(flattened_probs) * activation_bar_ratio)).values.item()
    # print((selected_probs > activation_bar).sum(dim=1).tolist())
    lang, indice = torch.where(selected_probs > activation_bar)

    merged_index = torch.stack((row_index, col_index), dim=-1)
    final_indice = []
    for _, index in enumerate(indice.split(torch.bincount(lang).tolist())):
        lang_index = [tuple(row.tolist()) for row in merged_index[index]]
        lang_index.sort()
        layer_index = [[] for _ in range(num_layers)]
        for l, h in lang_index:
            layer_index[l].append(h)
        for l, h in enumerate(layer_index):
            layer_index[l] = torch.tensor(h).long()
        final_indice.append(layer_index)
    return final_indice

In [ ]:
def get_prob_for_lape(tensor, num_lang, num_layer, num_neurons):
    """
    num_neurons: intermediate layer (neurons in a layer)
    """
    full_languages_raw_values = (tensor.transpose(0,1)[0] > 0).half()
    probs = full_languages_raw_values.mean(dim=-2)
    probs.shape
    del full_languages_raw_values
    probs = probs.reshape(num_lang,num_layer,num_neurons)
    transposed_probs = probs.transpose(0,1).transpose(-1,-2)
    transposed_probs.shape
    return transposed_probs

def convert_to_global_indices(final_indice, intermediate_size):
    final_flattened = []
    
    for lang_idx, layers in enumerate(final_indice):  # Iterate over languages
        global_indices = []
        for layer_idx, hidden_units in enumerate(layers):  # Iterate over layers
            if hidden_units.numel() > 0:  # If there are selected neurons
                global_indices.extend((layer_idx * intermediate_size + hidden_units).tolist())
        
        final_flattened.append((global_indices))  
    
    return final_flattened



In [ ]:

def visualize_lape(full_languages_raw_values, num_layer, top_rate = 0.05,filter_rate=0.80,activation_bar_ratio=0.80): 
    num_lang, _, num_rows, num_neurons = full_languages_raw_values.shape
    
    per_layer = int(num_neurons/num_layer)
    transposed_probs = get_prob_for_lape(full_languages_raw_values, num_lang, num_layer, per_layer)
    lape = LAPE(transposed_probs, top_rate = top_rate,filter_rate=filter_rate,activation_bar_ratio=activation_bar_ratio)
    flattened_indices = convert_to_global_indices(lape, per_layer)
    activation_dict = dict()
    for i in range (num_lang):
        activation_dict[i] = flattened_indices[i]
    make_heatmap_neuron_overlap(activation_dict, num_lang, False, "jaccard", 1)
    return lape

#### 🐸Massive Qwen

Jaccard similarity LAPE neurons. Sedikit yang overlap karena LAPE language-specific dan kemungkinannya kecil untuk sharing between neurons

In [ ]:
lape_massive_qwen = visualize_lape(full_languages_raw_values, 24, top_rate = 0.02,filter_rate=0.80,activation_bar_ratio=0.80)

#### 🐸XCOPA GEMMA

In [ ]:
lape_xcopa_gemma = visualize_lape(full_languages_raw_values_xcopa_gemma, num_layer=26, top_rate = 0.01,filter_rate=0.80,activation_bar_ratio=0.80)

#### intervention LAPE


In [ ]:
import psutil

def check_cpu_memory():
    mem = psutil.virtual_memory()
    print(f"Total Memory: {mem.total / 1024**3:.2f} GB")
    print(f"Available Memory: {mem.available / 1024**3:.2f} GB")
    print(f"Used Memory: {mem.used / 1024**3:.2f} GB")
    print(f"Memory Usage: {mem.percent}%")

import torch

def clear_cuda_cache():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

clear_cuda_cache()
check_cpu_memory()
!nvidia-smi

In [ ]:
import torch

def clear_cuda_cache():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

clear_cuda_cache()

In [ ]:
def clean_hooks(infer_model):
    for i in range(len(infer_model.model.model.layers)):
        mlp = infer_model.model.model.layers[i].mlp
        mlp.act_fn._forward_hooks.clear()
def test_inference(infer_model, prompt, max_new_tokens):
    generated_text = infer_model.inference(prompt, max_new_tokens)
    # print(f"data: {prompt}")
            
    print(f"gen_text {generated_text}")
def set_activation_mlp(name, lape_lang, is_multiplied, replace_value): 
    """
        name (str): buat namain layer
        lape_lang: lape untuk suatu lang
        is_multiplied: (bool) true if multiplied else replaced
    """
    def hook_fn(module, input, output):
        # print(f"output {output.shape}")
        layer = int(name)
        if is_multiplied:
            output[0, :, lape_lang[layer]] *= replace_value
        else:
            output[0, :, lape_lang[layer]] = replace_value
    return hook_fn


def intervensi(prompt, infer_model, lape_lang, num_layers, max_new_tokens, is_multiplied, replace_value):
    clean_hooks(infer_model)
    handlers = []
    for i in range (num_layers):
        mlp = infer_model.model.model.layers[i].mlp
        handlers.append(mlp.act_fn.register_forward_hook(set_activation_mlp(f"{i}", lape_lang, is_multiplied, replace_value)))
    test_inference(infer_model, prompt, max_new_tokens)
    for i in handlers:
        i.remove()
    clean_hooks(infer_model)

def intervene_langs(infer_model, prompt_lang, lape_langs, num_layers, max_new_tokens=10, is_multiplied=False, replace_value=0):
    """
    given a prompt, from number of languages, see how intervention each language affects the prompt.
    infer_model: InferenceModel
    prompt_lang: (str) prompt dalam bahasa tertentu
    lape_langs: (dict) key: lang, value: lape in a language
    """
    print(f"original: ")
    test_inference(infer_model, prompt_lang, max_new_tokens)
    for key, value in lape_langs.items():
        print(f"intervensi {key}")
        intervensi(prompt_lang, infer_model, value, num_layers, max_new_tokens, is_multiplied, replace_value)


#### 🐸Massive Qwen

In [ ]:
infer_model = massive_qwen
num_layers = 24

In [ ]:
lape_langs = {
    "indo" : lape_massive_qwen[19],
    "malay" : lape_massive_qwen[31],
    'java' : lape_massive_qwen[23],
    'eng': lape_massive_qwen[10]
}
prompt_indo = "batalkan alarm saya pukul tujuh pagi"
intervene_langs(
    infer_model, prompt_indo, lape_langs, num_layers, max_new_tokens=10, is_multiplied=False, replace_value=-1)

In [ ]:
lape_langs = {
    "indo" : lape_massive_qwen[19],
    "malay" : lape_massive_qwen[31],
    'java' : lape_massive_qwen[23],
    'eng': lape_massive_qwen[10]
}
prompt_indo = "batalkan alarm saya pukul tujuh pagi"
intervene_langs(
    infer_model, prompt_indo, lape_langs, num_layers, max_new_tokens=10, is_multiplied=False, replace_value=-1)

In [ ]:
prompt_indo = "batalkan alarm saya pukul tujuh pagi"
intervene_langs(
    infer_model, prompt_indo, lape_langs, num_layers, max_new_tokens=10, is_multiplied=False, replace_value=0)

In [ ]:
prompt_indo = "batalkan alarm saya pukul tujuh pagi"
intervene_langs(
    massive_qwen, prompt_indo, lape_langs, num_layers, max_new_tokens=10, is_multiplied=True, replace_value=-1)

#### 🐸XCOPA Gemma

In [ ]:
# lape_massive_qwen

In [ ]:
infer_model = xcopa_gemma
num_layers = 26

In [ ]:
clean_hooks(xcopa_gemma)
xcopa_gemma.model.model.layers[0].mlp.act_fn._forward_hooks


In [ ]:
lape_langs = {
    "indo" : lape_xcopa_gemma[2],
    "haitian" : lape_xcopa_gemma[1],
    'chinese' : lape_xcopa_gemma[10],
    'swahili': lape_xcopa_gemma[5]
}
prompt_indo = "batalkan alarm saya pukul tujuh pagi"
intervene_langs(
    xcopa_gemma, prompt_indo, lape_langs, num_layers, max_new_tokens=20, is_multiplied=False, replace_value=0)

In [ ]:
clean_hooks(xcopa_gemma)
lape_langs = {
    "indo" : lape_xcopa_gemma[2],
    "haitian" : lape_xcopa_gemma[1],
    'chinese' : lape_xcopa_gemma[10],
    'swahili': lape_xcopa_gemma[5]
}
prompt_indo = "batalkan alarm saya pukul tujuh pagi"
intervene_langs(
    xcopa_gemma, prompt_indo, lape_langs, num_layers, max_new_tokens=20, is_multiplied=True, replace_value=-1)

=================================